In [ ]:
from datetime import timedelta
import cv2
import numpy as np
import os
from tqdm.notebook import tqdm
import glob

## Extract frames

In [ ]:
# extract frames with fixed fps: 
# https://www.thepythoncode.com/article/extract-frames-from-videos-in-python
SAVING_FRAMES_PER_SECOND = 10

# def format_timedelta(td: timedelta):
#     """Utility function to format timedelta objects in a cool way 
#     (from e.g 00:00:20.05 to 0-00-20.05) 
#     omitting microseconds and retaining milliseconds"""
#     result = str(td)
#     try:
#         result, ms = result.split(".") # if td is not whole seconds
#     except ValueError:
#         return (result + ".00").replace(":", "-")
#     ms = int(ms)
#     ms = round(ms / 1e4)
#     return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

def extract_frames(video_file, save_fps=SAVING_FRAMES_PER_SECOND):
    basename = os.path.dirname(video_file)
    filename, _ = os.path.splitext(os.path.basename(video_file))
#     filename += "-opencv"
    foldername  = "-".join( filename.split("-")[:2] ) # use the youtube title +id as folder name
    foldername = os.path.join(basename, foldername)
    print("save to :",foldername)
    # make a folder by the name of the video file
    if not os.path.isdir(foldername):
        os.mkdir(foldername)
    # read the video file   
    assert os.path.isfile(video_file)
    cap = cv2.VideoCapture(video_file)
    print(f"#frames:{cap.get(cv2.CAP_PROP_FRAME_COUNT)}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, "
          f"height:{cap.get(cv2.CAP_PROP_FRAME_HEIGHT)}, width:{cap.get(cv2.CAP_PROP_FRAME_WIDTH)}")
    
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, save_fps)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    print(f"Total saving frames: {len(saving_frames_durations)}")
    
    # start the loop
    frame_count = 0
    save_count = 0
    with tqdm(total=len(saving_frames_durations)) as pbar:
        while len(saving_frames_durations): # if the list is empty, all duration frames were saved
            is_read, frame = cap.read()
            if not is_read:
                break # break out of the loop if there are no frames to read

            # get the duration by dividing the frame count by the FPS
            frame_duration = frame_count / fps
            frame_count += 1 # increment the frame count
            closest_duration = saving_frames_durations[0] # get the earliest duration to save
            if frame_duration >= closest_duration:
                # if closest duration is less than/equals the frame duration, then save the frame
#                 frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
#                 cv2.imwrite(os.path.join(foldername, f"frame_{frame_duration_formatted}.jpg"), frame) 
                cv2.imwrite(os.path.join(foldername, f"{save_count}.jpg"), frame) 
                saving_frames_durations.pop(0)
                save_count += 1
                pbar.update()

In [ ]:
video_file = "origami_for_beginners_Bird_Pigeon-L6ciLmiEfg0-854x480_30fps.mp4"
extract_frames(video_file)

save to : origami_for_beginners_Bird_Pigeon-L6ciLmiEfg0
#frames:6312.0, FPS: 29.97002997002997, height:480.0, width:854.0
Total saving frames: 2107



 84%|████████████████████████████████▌      | 1760/2107 [00:22<00:04, 84.95it/s]


100%|██████████████████████████████████████▉| 2106/2107 [00:25<00:00, 81.84it/s]


## Create label

dove:

    0: other
    1: white paper
    2: blue triangle
    3: 2 w/ a smaller triangle folded over
    4: 3 w/ a white triangle folded back to the left
    5: 4 w/ the top half folded down
    6: 5 w/ one wing folded up
    7: flip over
    8: 7 w/ the other wing folded up
    9: 8 flipped over
    10: 9 + head insided inverse fold
    11: finish
    -1: end

In [ ]:
video_file = "origami_for_beginners_Bird_Pigeon-L6ciLmiEfg0-854x480_30fps.mp4"
basename = os.path.dirname(video_file)
filename, _ = os.path.splitext(os.path.basename(video_file))
foldername  = "-".join( filename.split("-")[:2] ) # use the youtube title +id as folder name
foldername = os.path.join(basename, foldername)
n_frames = len(glob.glob(os.path.join(foldername+"/*.jpg")))

# for origami_for_beginners_Bird_Pigeon-L6ciLmiEfg0-854x480_30fps
label_interval = {0:11, 76:0, 97:1, 118:0, 129:1, 174:0, 182:1, 190:0, 342:2, 509:0, 624:3, 769:0, 894:4, 
                  1029:0, 1175:5, 1192:0, 1229:5, 1348:0, 1439:6, 1467:0, 1473:7, 1513:0, 1583:8, 1584:0, 
                  1596: 8, 1601: 0, 1609: 9, 1650:0, 1760:10, 1764:0, 1770:10, 1775:0, 1906:11, 2106:-1}
labels = []
label_interval = sorted(label_interval.items())
for i in range(len(label_interval) - 1):
    curr_time, curr_label = label_interval[i]
    next_time, _ = label_interval[i+1]
    labels += [curr_label] * (next_time - curr_time)
print("total labels: ", len(labels))
assert len(labels) == n_frames

labels = np.asarray(labels)
np.savetxt(os.path.join(foldername, "label.txt"), labels, fmt="%d")

total labels:  2106


In [ ]:
t = np.loadtxt(foldername+"/label.txt")
t.shape

(2106,)